In [1]:
import numpy as np
import pandas as pd
import mne
import csv

# Data Loading

### Loading Features

In [2]:
# # File path of loading file
# feature_path = './save/features_all.csv'
# label_path = './save/labels.csv'

In [3]:
# # Load features from CSV file
# features = np.loadtxt(feature_path, delimiter=',')

# # Load labels from CSV file using Python's csv module
# labels = []
# with open(label_path, 'r') as csvfile:
#     csvreader = csv.reader(csvfile)
#     next(csvreader)  # Skip the header row
#     for row in csvreader:
#         labels.append(int(row[0]))

# labels = np.array(labels)

In [4]:
# print(f'features: {features.shape}, labels: {labels.shape}')

### Loading Epochs

In [26]:
# Load saved data from CSV
features = np.load('./save/dataset_epochs/epochs_data.npy')

# Load labels from CSV
labels_df = pd.read_csv('./save/dataset_epochs/labels.csv')
labels = labels_df['labels'].values

In [27]:
print(f'features: {features.shape}, labels: {labels.shape}')

features: (286, 22, 1001), labels: (286,)


# Customize Dataset

In [7]:
import torch
from dataset.dataset import EEGDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [9]:
# Convert your data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

In [23]:
import torch.nn.functional as F
y_train_one_hot = F.one_hot(y_train_tensor - 7, num_classes=4).float()

In [10]:
# Convert your data to PyTorch tensors
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [24]:
# Create an instance of the custom dataset
train_dataset = EEGDataset(X_train_tensor, y_train_one_hot)
test_dataset = EEGDataset(X_test_tensor, y_test_tensor)

# Create a DataLoader for training
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/home/seohyunseo/eeg_signal_processing_v2/dataset/dataset.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float32)
/home/seohyunseo/eeg_signal_processing_v2/dataset/dataset.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


# Loading EEGNet model

In [12]:
import torch.nn as nn
from models.EEGNet import EEGNet
from models.GRU import GRUModel
from models.BasicNet import BasicNet

In [13]:
# # Initialize the model, loss function, and optimizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print('torch version: ', torch.__version__)
# print('GPU State:', device)

In [14]:
# EEGNet using Pytorch
class EEGNet(nn.Module):
    def __init__(self, num_classes=4, AF=nn.ELU(alpha=1)):
        super(EEGNet, self).__init__()
        self.firstConv = nn.Sequential(
            nn.Conv2d(1, 16, (1, 51), stride=(1, 1), padding=(0, 25), bias=False),
            nn.BatchNorm2d(16, 1e-05)
        )      
        self.depthwiseConv = nn.Sequential(
            nn.Conv2d(16, 32, (2, 1), stride=(1, 1), groups=16, bias=False),
            nn.BatchNorm2d(32, 1e-05),
            AF,
            nn.AvgPool2d(kernel_size=(1,4), stride=(1,4), padding=0),
            nn.Dropout(p=0.25)
        )
        self.separableConv = nn.Sequential(
            nn.Conv2d(32, 32, (1, 15), stride=(1, 1), padding=(0, 7), bias=False),
            nn.BatchNorm2d(32, 1e-05),
            AF,
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0),
            nn.Dropout(p=0.25)
        )
        self.classify = nn.Sequential(
            nn.Linear(in_features=20832, out_features=num_classes, bias=True)
        )
        
    def forward(self, x):
        x = self.firstConv(x)
        print(x.shape)
        x = self.depthwiseConv(x)
        print(x.shape)
        x = self.separableConv(x)
        print(x.shape)
        x = self.classify(x.view(len(x), -1))
        return x  

In [15]:
EEGNet()

EEGNet(
  (firstConv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (depthwiseConv): Sequential(
    (0): Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1)
    (3): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (4): Dropout(p=0.25, inplace=False)
  )
  (separableConv): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1)
    (3): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (4): Dropout(p=0.25, inplace=False)
  )
  (classify): Sequential(
    (0): Linear(in_features=20832, out_features=4, bias=True)
  )
)

In [16]:
import torch.optim as optim

In [18]:
model = EEGNet(num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model

In [25]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader :
        inputs, labels = inputs, labels

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        accuracy = total_correct / total_samples
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}")

torch.Size([32, 16, 22, 1001])
torch.Size([32, 32, 21, 250])
torch.Size([32, 32, 21, 31])


RuntimeError: Expected floating point type for target with class probabilities, got Long